In [1]:
# Load data from Excel file
def GetData(WordFile, WordWorksheet, GridFile, GridWorksheet):
    Rank = LoadFromExcel(WordFile, WordWorksheet, 'rank')
    Frequency = LoadFromExcel(WordFile, WordWorksheet, 'frequency')
    Word = LoadFromExcel(WordFile, WordWorksheet, 'word')
    Rank.columns = ['Candidate']
    Frequency.columns = ['Candidate']
    Word.columns = ['Candidate']
    
    GridWords = LoadFromExcel(GridFile, GridWorksheet, 'NumWords')
    NumIntersections = LoadFromExcel(GridFile, GridWorksheet, 'Intersections')
    AcrossRef = LoadFromExcel(GridFile, GridWorksheet, 'AcrossRef')
    AcrossPos = LoadFromExcel(GridFile, GridWorksheet, 'AcrossPos')
    DownRef = LoadFromExcel(GridFile, GridWorksheet, 'DownRef')
    DownPos = LoadFromExcel(GridFile, GridWorksheet, 'DownPos')
    UseFixWords = LoadFromExcel(GridFile, GridWorksheet, 'UseFixWords')
    FixWords = LoadFromExcel(GridFile, GridWorksheet, 'FixWords')
    
    return Rank, Frequency, Word, GridWords, NumIntersections, AcrossRef, AcrossPos, DownRef, DownPos, UseFixWords, FixWords

In [2]:
# Define model data, assigning all data to the Model
def DefineModelData(Model, Rank, Frequency, Word, GridWords, NumIntersections, AcrossRef, AcrossPos, DownRef, DownPos, UseFixWords, FixWords):

    Grid_rows, Grid_cols = np.shape(AcrossRef)   # Note: Model defined as h, w while data accessed via w, h

    Model.GridWords = pyo.Set(initialize = range(0, GridWords.iloc[0][0]))   # Set for number of words in the grid
    Model.NumIntersections = NumIntersections.iloc[0][0]
    Model.GridWidth = pyo.Set(initialize = range(0, Grid_cols))   # Set width of the grid
    Model.GridHeight = pyo.Set(initialize = range(0, Grid_rows))   # Set for the height of the grid
    Model.AcrossRef = pyo.Param(Model.GridHeight, Model.GridWidth, within = pyo.NonNegativeIntegers, mutable = True)
    Model.AcrossPos = pyo.Param(Model.GridHeight, Model.GridWidth, within = pyo.NonNegativeIntegers, mutable = True)
    Model.DownRef = pyo.Param(Model.GridHeight, Model.GridWidth, within = pyo.NonNegativeIntegers, mutable = True)
    Model.DownPos = pyo.Param(Model.GridHeight, Model.GridWidth, within = pyo.NonNegativeIntegers, mutable = True)
    Model.GridLengths = pyo.Param(Model.GridWords, within = pyo.NonNegativeIntegers, mutable = True, initialize = 0)

    for w in Model.GridWidth:
        for h in Model.GridHeight:
            Model.AcrossRef[h, w] = AcrossRef[w][h]   # Populate grid encoding
            Model.AcrossPos[h, w] = AcrossPos[w][h]
            Model.DownRef[h, w] = DownRef[w][h]
            Model.DownPos[h, w] = DownPos[w][h]
            if AcrossRef[w][h] >= 1:   # Get length of "across" words by looking at maximum position of each word
                Model.GridLengths[AcrossRef[w][h] - 1] = max(pyo.value(Model.GridLengths[AcrossRef[w][h] - 1]), AcrossPos[w][h])
            if DownRef[w][h] >= 1:   # Get length of "down" words by looking at maximum position of each word
                Model.GridLengths[DownRef[w][h] - 1] = max(pyo.value(Model.GridLengths[DownRef[w][h] - 1]), DownPos[w][h])

    SelectedWords = []   # Select words as candidates only if they match the grid's word lengths
    GridWordLengths = []   # Length of words in the grid
    for g in Model.GridLengths:
        if pyo.value(Model.GridLengths[g]) not in GridWordLengths:
            GridWordLengths.append(pyo.value(Model.GridLengths[g]))
    GridWordLengths.sort()   # e.g. [4,5,9,15]
    for c in range(0, len(Word)):
        if len(Word['Candidate'][c]) in GridWordLengths:
            SelectedWords.append(c)   # Subset of lexicon words, included only if they fit in the grid

    Size = SampleSize
    if Size == 0:
        Size = len(SelectedWords)
    else:
        Size = min(SampleSize, len(SelectedWords))
    print(f'Lexicon size: {Size:,.0f} (out of {len(SelectedWords):,.0f} that fit, from lexicon of {len(Word):,.0f})')   # After excluding words that don't fit in the grid
    
    Model.Candidate = pyo.Set(initialize = range(0, Size))   # Set of candidate words 
    Model.Ascii = pyo.Set(initialize = range(0, MaxWordLength))   # Set of ASCII character codes in a word
    Model.Rank = pyo.Param(Model.Candidate, within = pyo.NonNegativeIntegers, mutable = True)   # Lexicon rank
    Model.Frequency = pyo.Param(Model.Candidate, within = pyo.NonNegativeReals, mutable = True)   # Lexicon frequency
    Model.Length = pyo.Param(Model.Candidate, within = pyo.NonNegativeIntegers, mutable = True)   # Number of characters in each candidate word
    Model.Word = pyo.Param(Model.Candidate, Model.Ascii, within = pyo.NonNegativeIntegers, mutable = True)   # Each candidate word, split into ASCII codes
    
    List = [i for i in range(0, len(SelectedWords))]   # Row numbers for whole lexicon
    if Size == len(SelectedWords):
        Sample = List   # Use whole lexicon
    else:
        Sample = rnd.sample(List, Size)   # Use sample of lexicon
    CandidateNum = 0
    for c in range(0, len(SelectedWords)):   # Populate data for selected sample words
        if c in Sample:
            Model.Rank[CandidateNum] = Rank['Candidate'][SelectedWords[c]]
            Model.Frequency[CandidateNum] = Frequency['Candidate'][SelectedWords[c]]
            Model.Length[CandidateNum] = len(Word['Candidate'][SelectedWords[c]])
            LettersAscii = list(bytes(Word['Candidate'][SelectedWords[c]], 'ascii'))   # ASCII codes for word's characters
            for a in Model.Ascii:   # list of ASCII codes for each letter of a word, padded with zeroes beyond the word's length
                if a >= len(LettersAscii):
                    Model.Word[CandidateNum, a] = 0
                else:
                    Model.Word[CandidateNum, a] = LettersAscii[a]
            CandidateNum += 1
                
    if UseFixWords[0][0] == False:   # Data indicating whether to use the list of word positions to fix
        NumFixWords = 0   # Do not fix words
    else:
        NumFixWords = len(FixWords)   # Or number of words to fix
    Model.FixGridWord = pyo.Param(range(0, NumFixWords), within = pyo.NonNegativeIntegers, mutable = True, initialize = 0)
    for f in range(0, NumFixWords):   # Populate list of word positions to fix, if any
        Model.FixGridWord[f] = FixWords[0][f]